In [ ]:
!pip install k3im --upgrade

import os

os.environ['KERAS_BACKEND'] = 'jax'

In [5]:
import numpy as np
import keras


def readucr(filename):
    data = np.loadtxt(filename, delimiter="\t")
    y = data[:, 0]
    x = data[:, 1:]
    return x, y.astype(int)


root_url = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"

x_train, y_train = readucr(root_url + "FordA_TRAIN.tsv")
x_test, y_test = readucr(root_url + "FordA_TEST.tsv")

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

n_classes = len(np.unique(y_train))

idx = np.random.permutation(len(x_train))
x_train = x_train[idx]
y_train = y_train[idx]

y_train[y_train == -1] = 0
y_test[y_test == -1] = 0

In [6]:
x_train.shape

(3601, 500, 1)

In [7]:
def train_model(model):
    model.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=keras.optimizers.Adam(learning_rate=1e-4),
        metrics=["sparse_categorical_accuracy"],
    )


    model.fit(
        x_train,
        y_train,
        validation_split=0.2,
        epochs=2,
        batch_size=64,
    )
    model.evaluate(x_test, y_test, verbose=1)

In [8]:
from k3im.cct_1d import CCT_1DModel
model = CCT_1DModel(
    input_shape=(500, 1),
    num_heads=4,
    projection_dim=154,
    kernel_size=10,
    stride=15,
    padding=5,
    transformer_units=[154],
    stochastic_depth_rate=0.5,
    transformer_layers=1,
    num_classes=n_classes,
    positional_emb=False,
)

CUDA backend failed to initialize: Found CUDA version 11070, but JAX was built against version 11080, which is newer. The copy of CUDA that is installed must be at least as new as the version against which JAX was built. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [9]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 500, 1)    │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ cct_tokenizer1d     │ (None, 6, 154)    │  99,200 │ input_layer[0][0]    │
│ (CCTTokenizer1D)    │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalization │ (None, 6, 154)    │     308 │ cct_tokenizer1d[0][… │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ multi_head_attenti… │ (None, 6, 154)    │ 381,458 │ layer_normalization… │
│ (MultiHeadAttentio… │                   │         │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ stochastic_depth    │ (None, 6, 154)    │       0 │ multi_head_attentio… │
│ (StochasticDepth)   │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add (Add)           │ (None, 6, 154)    │       0 │ stochastic_depth[0]… │
│                     │                   │         │ cct_tokenizer1d[0][… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 6, 154)    │     308 │ add[0][0]            │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense (Dense)       │ (None, 6, 154)    │  23,870 │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dropout_1 (Dropout) │ (None, 6, 154)    │       0 │ dense[0][0]          │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ stochastic_depth_1  │ (None, 6, 154)    │       0 │ dropout_1[0][0]      │
│ (StochasticDepth)   │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_1 (Add)         │ (None, 6, 154)    │       0 │ stochastic_depth_1[… │
│                     │                   │         │ add[0][0]            │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 6, 154)    │     308 │ add_1[0][0]          │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ sequence_pooling    │ (None, 154)       │     155 │ layer_normalization… │
│ (SequencePooling)   │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_2 (Dense)     │ (None, 2)         │     310 │ sequence_pooling[0]… │
└─────────────────────┴───────────────────┴─────────┴──────────────────────┘

 Total params: 505,917 (1.93 MB)

 Trainable params: 505,917 (1.93 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
train_model(model)

Epoch 1/2
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - loss: 0.7615 - sparse_categorical_accuracy: 0.5196 - val_loss: 0.6368 - val_sparse_categorical_accuracy: 0.6200
Epoch 2/2
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.6129 - sparse_categorical_accuracy: 0.6501 - val_loss: 0.5908 - val_sparse_categorical_accuracy: 0.6602
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.5311 - sparse_categorical_accuracy: 0.7191 


In [11]:
from k3im.convmixer_1d import ConvMixer1DModel

In [12]:
from k3im.convmixer_1d import ConvMixer1DModel
model = ConvMixer1DModel(seq_len=500,
    n_features=1,
    filters=128,
    depth=4,
    kernel_size=15,
    patch_size=4,
    num_classes=n_classes,)

In [13]:
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 500, 1)    │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 125, 128)  │     640 │ input_layer_2[0][0]  │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ activation          │ (None, 125, 128)  │       0 │ conv1d_2[0][0]       │
│ (Activation)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ batch_normalization │ (None, 125, 128)  │     512 │ activation[0][0]     │
│ (BatchNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ depthwise_conv1d    │ (None, 125, 128)  │   2,048 │ batch_normalization… │
│ (DepthwiseConv1D)   │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ activation_1        │ (None, 125, 128)  │       0 │ depthwise_conv1d[0]… │
│ (Activation)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ batch_normalizatio… │ (None, 125, 128)  │     512 │ activation_1[0][0]   │
│ (BatchNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_2 (Add)         │ (None, 125, 128)  │       0 │ batch_normalization… │
│                     │                   │         │ batch_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 125, 128)  │  16,512 │ add_2[0][0]          │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ activation_2        │ (None, 125, 128)  │       0 │ conv1d_3[0][0]       │
│ (Activation)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ batch_normalizatio… │ (None, 125, 128)  │     512 │ activation_2[0][0]   │
│ (BatchNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ depthwise_conv1d_1  │ (None, 125, 128)  │   2,048 │ batch_normalization… │
│ (DepthwiseConv1D)   │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ activation_3        │ (None, 125, 128)  │       0 │ depthwise_conv1d_1[… │
│ (Activation)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ batch_normalizatio… │ (None, 125, 128)  │     512 │ activation_3[0][0]   │
│ (BatchNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_3 (Add)         │ (None, 125, 128)  │       0 │ batch_normalization… │
│                     │                   │         │ batch_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv1d_4 (Conv1D)   │ (None, 125, 128)  │  16,512 │ add_3[0][0]          │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ activation_4        │ (None, 125, 128)  │       0 │ conv1d_4[0][0]       │
│ (Activation)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ batch_normalizatio… │ (None, 125, 128)  │     512 │ activation_4[0][0] 

 Total params: 79,746 (311.51 KB)

 Trainable params: 77,442 (302.51 KB)

 Non-trainable params: 2,304 (9.00 KB)

In [14]:
train_model(model)

Epoch 1/2
45/45 ━━━━━━━━━━━━━━━━━━━━ 33s 672ms/step - loss: 0.5019 - sparse_categorical_accuracy: 0.8043 - val_loss: 0.8098 - val_sparse_categorical_accuracy: 0.4854
Epoch 2/2
45/45 ━━━━━━━━━━━━━━━━━━━━ 30s 661ms/step - loss: 0.2249 - sparse_categorical_accuracy: 0.9216 - val_loss: 1.0526 - val_sparse_categorical_accuracy: 0.4854
42/42 ━━━━━━━━━━━━━━━━━━━━ 6s 128ms/step - loss: 1.0772 - sparse_categorical_accuracy: 0.4701


In [17]:
from k3im.eanet_1d import EANet1DModel
model = EANet1DModel(
    seq_len=500,
    patch_size=20,
    num_classes=n_classes,
    dim=96,
    depth=3,
    heads=32,
    mlp_dim=64,
    dim_coefficient=2,
    attention_dropout=0.0,
    channels=1,
)

In [18]:
model.summary()

Model: "functional_26"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_18      │ (None, 500, 1)    │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reshape_32          │ (None, 25, 20)    │       0 │ input_layer_18[0][0] │
│ (Reshape)           │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 25, 20)    │      40 │ reshape_32[0][0]     │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_92 (Dense)    │ (None, 25, 96)    │   2,016 │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 25, 96)    │     192 │ dense_92[0][0]       │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_32 (Add)        │ (None, 25, 96)    │       0 │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_93 (Dense)    │ (None, 25, 192)   │  18,624 │ add_32[0][0]         │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reshape_33          │ (None, 25, 64, 3) │       0 │ dense_93[0][0]       │
│ (Reshape)           │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ transpose_28        │ (None, 64, 25, 3) │       0 │ reshape_33[0][0]     │
│ (Transpose)         │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_94 (Dense)    │ (None, 64, 25,    │     192 │ transpose_28[0][0]   │
│                     │ 48)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ softmax_14          │ (None, 64, 25,    │       0 │ dense_94[0][0]       │
│ (Softmax)           │ 48)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ lambda_14 (Lambda)  │ (None, 64, 25,    │       0 │ softmax_14[0][0]     │
│                     │ 48)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dropout_28          │ (None, 64, 25,    │       0 │ lambda_14[0][0]      │
│ (Dropout)           │ 48)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_95 (Dense)    │ (None, 64, 25, 3) │     147 │ dropout_28[0][0]     │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ transpose_29        │ (None, 25, 64, 3) │       0 │ dense_95[0][0]       │
│ (Transpose)         │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reshape_34          │ (None, 25, 192)   │       0 │ transpose_29[0][0]   │
│ (Reshape)           │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_96 (Dense)    │ (None, 25, 96)    │  18,528 │ reshape_34[0][0]     │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dropout_29          │ (None, 25, 96)    │       0 │ dense_96[0][0]       │
│ (Dropout)           │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼────────────────────

 Total params: 153,027 (597.76 KB)

 Trainable params: 153,027 (597.76 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
train_model(model)

Epoch 1/2
45/45 ━━━━━━━━━━━━━━━━━━━━ 10s 159ms/step - loss: 0.6854 - sparse_categorical_accuracy: 0.5401 - val_loss: 0.6545 - val_sparse_categorical_accuracy: 0.6657
Epoch 2/2
45/45 ━━━━━━━━━━━━━━━━━━━━ 6s 143ms/step - loss: 0.6384 - sparse_categorical_accuracy: 0.6693 - val_loss: 0.6164 - val_sparse_categorical_accuracy: 0.7074
42/42 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 0.6006 - sparse_categorical_accuracy: 0.7457


In [23]:
from k3im.gmlp_1d import gMLP1DModel
model = gMLP1DModel(seq_len=500, patch_size=20, num_classes=n_classes, dim=64, depth=4, channels=1, dropout_rate=0.0)

In [24]:
model.summary()

Model: "functional_38"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ input_layer_27 (InputLayer)     │ (None, 500, 1)            │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ reshape_40 (Reshape)            │ (None, 25, 20)            │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ layer_normalization_42          │ (None, 25, 20)            │         40 │
│ (LayerNormalization)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_126 (Dense)               │ (None, 25, 64)            │      1,344 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ layer_normalization_43          │ (None, 25, 64)            │        128 │
│ (LayerNormalization)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ add_40 (Add)                    │ (None, 25, 64)            │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ g_mlp_layer_4 (gMLPLayer)       │ (None, 25, 64)            │     13,386 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ g_mlp_layer_5 (gMLPLayer)       │ (None, 25, 64)            │     13,386 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ g_mlp_layer_6 (gMLPLayer)       │ (None, 25, 64)            │     13,386 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ g_mlp_layer_7 (gMLPLayer)       │ (None, 25, 64)            │     13,386 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ avg_pool                        │ (None, 64)                │          0 │
│ (GlobalAveragePooling1D)        │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_139 (Dense)               │ (None, 2)                 │        130 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 55,186 (215.57 KB)

 Trainable params: 55,186 (215.57 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
train_model(model)

Epoch 1/2
45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - loss: 0.7124 - sparse_categorical_accuracy: 0.5276 - val_loss: 0.6737 - val_sparse_categorical_accuracy: 0.5784
Epoch 2/2
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.6635 - sparse_categorical_accuracy: 0.6002 - val_loss: 0.6409 - val_sparse_categorical_accuracy: 0.6449
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.6379 - sparse_categorical_accuracy: 0.6455


In [26]:
from k3im.mlp_mixer_1d import Mixer1DModel
model = Mixer1DModel(seq_len=500, patch_size=20, num_classes=n_classes, dim=64, depth=4, channels=1, dropout_rate=0.0)

In [27]:
train_model(model)

Epoch 1/2
45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - loss: 0.8580 - sparse_categorical_accuracy: 0.5061 - val_loss: 0.7228 - val_sparse_categorical_accuracy: 0.5021
Epoch 2/2
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.7094 - sparse_categorical_accuracy: 0.5196 - val_loss: 0.6911 - val_sparse_categorical_accuracy: 0.5395
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.6815 - sparse_categorical_accuracy: 0.5537


In [29]:
from k3im.simple_vit_1d import SimpleViT1DModel
model = SimpleViT1DModel(seq_len=500,
    patch_size=20,
    num_classes=n_classes,
    dim=32,
    depth=3,
    heads=8,
    mlp_dim=64,
    channels=1,
    dim_head=64)

In [30]:
train_model(model)

Epoch 1/2
45/45 ━━━━━━━━━━━━━━━━━━━━ 6s 59ms/step - loss: 1.0425 - sparse_categorical_accuracy: 0.4848 - val_loss: 0.6978 - val_sparse_categorical_accuracy: 0.5173
Epoch 2/2
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - loss: 0.6853 - sparse_categorical_accuracy: 0.5477 - val_loss: 0.6752 - val_sparse_categorical_accuracy: 0.5659
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.6637 - sparse_categorical_accuracy: 0.5900


In [31]:
from k3im.vit_1d import ViT1DModel
model = ViT1DModel(seq_len=500,
    patch_size=20,
    num_classes=n_classes,
    dim=32,
    depth=3,
    heads=8,
    mlp_dim=64,
    channels=1,
    dim_head=64)

In [32]:
train_model(model)

Epoch 1/2
45/45 ━━━━━━━━━━━━━━━━━━━━ 6s 62ms/step - loss: 0.6974 - sparse_categorical_accuracy: 0.5314 - val_loss: 0.6330 - val_sparse_categorical_accuracy: 0.6227
Epoch 2/2
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - loss: 0.6143 - sparse_categorical_accuracy: 0.6423 - val_loss: 0.5850 - val_sparse_categorical_accuracy: 0.6644
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.5802 - sparse_categorical_accuracy: 0.6854
